# TRANSIT analysis of Mendadione TN-seq data

Jennifer J. Stiens

j.j.stiens@gmail.com

04.08.2023

[TRANSIT docs](https://transit.readthedocs.io/en/latest/method_HMM.html)

## Run TRANSIT HMM on each condition

1. Normalise all datasets with TTP
2. Aggregate data from each condition into single wig file ('agg_menplus.wig' and 'agg_menminus.wig')
3. Run TRANSIT HMM on each condition

New version (v3.2.7) available which has new resampling program. Try to install inside conda env

In [ ]:
#try installing inside conda env with pip
#change conda env
conda activate python38
pip install tnseq-transit
#ERROR: Failed building wheel for wxPython
# do this first:
mamba install -c conda-forge wxpython

#success!
transit --version
#3.3.0


going forward use python38 conda env for transit

In [ ]:
#don't run

#try installing from github repo
# https://github.com/mad-lab/transit/
conda install git pip
pip install git+https://github.com/mad-lab/transit.git
#same error


In [ ]:
#don't run

#create .yaml file and try new conda env (not sure where to put this file--in men_tnseq folder?)
conda env create -f transit.yaml
# some kind of error reported to conda


This appears to have created a new conda env called 'transit'. However do not see transit listed as a package.

Normalise datasets with TTR 
The command line instructions in docs are wrong for new version. Requires 'combined wig' vs comma-separated list, and no prot table

Need to look at old documentation to verify what combined wig format includes

[old transit docs](https://transit.readthedocs.io/en/v3.2.3/transit_methods.html)

"Note: the combined_wig input file can be generated from multiple wig files through the Transit GUI (File->Export->Selected_Datasets->Combined_wig), or via the ‘export’ command on the command-line (see combined_wig)."

In [ ]:
#create combined wigs for each condition and for each library for each condition and for all conditions/reps (may not be exactly correct format for transit)

def combined_wig(wig_list, output_file):
    import pandas as pd
    """Combine multiple wig files into one wig file"""
    with open(output_file, 'w') as outfile:
        wig_df = pd.DataFrame()
        for wig in wig_list:
            #get sample name from wig file name
            sample = wig.split('/')[1].split('_')[0]
            with open(wig) as infile:
                #read in wig file
                if wig_df.empty==True:
                    wig_df = pd.read_csv(infile, sep=' ', header=0, comment='#', names=['pos', sample])
                else:
                    read_df = pd.read_csv(infile, sep=' ', header=0, comment='#', names=['pos', sample])
                    read_values = read_df[sample].to_list()
                    #add to df
                    wig_df.insert(loc=len(wig_df.columns), column=sample, value=read_values, allow_duplicates=True)
                    wig_df.head()
        #write to file
        wig_df.to_csv(outfile, sep=' ', header=True, index=False)
    return wig_df

def main():
    import glob
    #list files in directory with .wig extension
    wig_files = glob.glob("output" + "/*.wig")
    my_wigs = ['output/A2_insertions.wig', 'output/C2_insertions.wig']
    combined_file = 'output/exp2_combined.wig'
    print(combined_wig(my_wigs, combined_file))

if __name__ == '__main__':
    main()


Use transit function to normalise and export in one go to start with--can use metadata to indicate libraries and conditions in resampling commands later

In [ ]:

#for all together
!transit export combined_wig output/A1_insertions.wig,output/C1_insertions.wig,output/A2_insertions.wig,output/C2_insertions.wig,output/B1_insertions.wig,output/D1_insertions.wig,output/B2_insertions.wig,output/D2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_samples_TTR.wig

#nonorm combined wig
!transit export combined_wig output/A1_insertions.wig,output/C1_insertions.wig,output/A2_insertions.wig,output/C2_insertions.wig,output/B1_insertions.wig,output/D1_insertions.wig,output/B2_insertions.wig,output/D2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_samples_nonorm.wig -n nonorm


# for each library
!transit export combined_wig output/A1_insertions.wig,output/C1_insertions.wig,output/B1_insertions.wig,output/D1_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_exp1_TTR.wig
!transit export combined_wig output/A2_insertions.wig,output/C2_insertions.wig,output/B2_insertions.wig,output/D2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_exp2_TTR.wig
#nonorm
!transit export combined_wig output/A1_insertions.wig,output/C1_insertions.wig,output/B1_insertions.wig,output/D1_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_exp1_nonorm.wig -n nonorm
!transit export combined_wig output/A2_insertions.wig,output/C2_insertions.wig,output/B2_insertions.wig,output/D2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_exp2_nonorm.wig -n nonorm

# for each condition
!transit export combined_wig output/A1_insertions.wig,output/C1_insertions.wig,output/A2_insertions.wig,output/C2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_menPos_TTR.wig
!transit export combined_wig output/B1_insertions.wig,output/D1_insertions.wig,output/B2_insertions.wig,output/D2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_menNeg_TTR.wig


In [73]:

#nonorm by condition
!transit export combined_wig output/A1_insertions.wig,output/C1_insertions.wig,output/A2_insertions.wig,output/C2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_menPos_nonorm.wig -n nonorm
!transit export combined_wig output/B1_insertions.wig,output/D1_insertions.wig,output/B2_insertions.wig,output/D2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_menNeg_nonorm.wig -n nonorm


[combined_wig] Starting Combined Wig Export
[combined_wig] Getting Data
[combined_wig] Normalizing
[combined_wig] Running Export Method...  99.3%   
[combined_wig] Finished Export
[combined_wig] Starting Combined Wig Export
[combined_wig] Getting Data
[combined_wig] Normalizing
[combined_wig] Running Export Method...  99.3%   
[combined_wig] Finished Export


## Run TRANSIT HMM on each condition to get essentiality calls

Do we do this on all 4 datasets, 2 libraries separately, or pool libraries post-normalisation? What to do with reps? 

transit on all 4 datasets will treat like 4 reps of one library, averaging or summing read counts from all datasets at each position
in DeJesus, pooled libraries to 'aggregate' data--choose best rep from each, or pool all of them? Pooling is essentially summing all the read counts instead of mean in transit. Better if uneven insertion density. Should be equivalent to adding in all 4 condition-specific datasets into transit hmm and selecting -r Sum (default is mean)

Normalised for each condition, pooled, normalised with transit again

In [29]:
#pool normalised insertions for all reps and both libraries in each condition (can just list as reps and transit will pool)
import pandas as pd
def pool_wigs(combined_wig, output_file):
    """
        Pool normalised insertions for all datasets in a combined wig file

    """
    import pandas as pd
    pooled_df = pd.DataFrame()
    wig_df = pd.read_csv(combined_wig, sep='\t', comment='#', header=None)
    pooled_df['TA_coord'] = wig_df[0]
    cols_to_sum = wig_df.columns[1:5]
    pooled_df['pooled'] = wig_df[cols_to_sum].sum(axis=1)
    pooled_df['gene'] = wig_df[5]
    header = "#" + combined_wig + "\n" + "#" + "pooled insertions" + "\n"
    f = open(output_file, 'a')
    f.write(header)
    pooled_df.to_csv(f, sep='\t', index=False, header=False)
    f.close()
    return pooled_df
    
pool_wigs("output/combined_menNeg_TTR.wig", "output/pooled_neg.wig")
pool_wigs("output/combined_menPos_TTR.wig", "output/pooled_pos.wig")


,TA_coord,pooled,gene
0,60,0.0,MB0001 (dnaA)
1,72,0.0,MB0001 (dnaA)
2,102,0.0,MB0001 (dnaA)
3,188,0.0,MB0001 (dnaA)
4,246,0.0,MB0001 (dnaA)
...,...,...,...
73531,4349793,38.2,NaN
73532,4349812,0.0,NaN
73533,4349851,0.0,NaN
73534,4349880,0.0,NaN


In [35]:
# what is insertion density of pooled
neg_df = pool_wigs("output/combined_menNeg_TTR.wig", "output/pooled_neg.wig")
pos_df = pool_wigs("output/combined_menPos_TTR.wig", "output/pooled_pos.wig")

def insertion_density(df):
    no_ins = len(df[df["pooled"] != 0])
    tot_tas = len(df)
    return round(no_ins/tot_tas,2)

if __name__ == "__main__":
    print(insertion_density(neg_df))
    print(insertion_density(pos_df))

0.63
0.53


In [ ]:
#run transit hmm on pooled wig files (will normalise again with ttr)

!transit hmm output/pooled_neg.wig ref_seqs/Mbovis_LT708304.prot_table output/transit/transit_hmm_neg
!transit hmm output/pooled_pos.wig ref_seqs/Mbovis_LT708304.prot_table output/transit/transit_hmm_pos

In [72]:
!head -n 4 output/transit/transit_hmm_pos_genes.txt
!head -n 4 output/transit/transit_hmm_neg_genes.txt

#HMM - Genes
#command line: python3 /Users/jenniferstiens/anaconda3/envs/python38/bin/transit hmm output/pooled_pos.wig ref_seqs/Mbovis_LT708304.prot_table transit_hmm_pos
#summary of gene calls: ES=483, GD=167, NE=3361, GA=24, N/A=10
#key: ES=essential, GD=insertions cause growth-defect, NE=non-essential, GA=insertions confer growth-advantage, N/A=not analyzed (genes with 0 TA sites)


#HMM - Genes
#command line: python3 /Users/jenniferstiens/anaconda3/envs/python38/bin/transit hmm output/pooled_neg.wig ref_seqs/Mbovis_LT708304.prot_table transit_hmm_neg
#summary of gene calls: ES=455, GD=141, NE=3417, GA=22, N/A=10
#key: ES=essential, GD=insertions cause growth-defect, NE=non-essential, GA=insertions confer growth-advantage, N/A=not analyzed (genes with 0 TA sites)


More genes are essential in the menadione positive condition.

In [101]:
#how do essential genes compare (naive comparison) with pooled normalised reps
import pandas as pd
def read_transit(transit_genes_file):   
    with open(transit_genes_file) as f:
        data = pd.read_csv(f, sep='\t', header=None, comment='#')
        data.columns = ['ORF', 'gene', 'annot', 'TAs', 'ES_sites', 'GD_sites', 'NE_sites', 'GA_sites', 'saturation', 'mean', 'call']
        es_genes = data[data['call'] == 'ES']
        es_orfs  = es_genes['ORF'].tolist()
        gd_genes = data[data['call'] == 'GD']
        gd_orfs  = gd_genes['ORF'].tolist()
        ne_genes = data[data['call'] == 'NE']
        ne_orfs  = ne_genes['ORF'].tolist()
        ga_genes = data[data['call'] == 'GA']
        ga_orfs  = ga_genes['ORF'].tolist()
    return es_orfs, gd_orfs, ne_orfs, ga_orfs

pos_genes = read_transit('output/transit/transit_hmm_pos_genes.txt')
neg_genes = read_transit('output/transit/transit_hmm_neg_genes.txt')

pos_es = pos_genes[0] 
neg_es = neg_genes[0]

#how many genes are essential in both?
both_ess = set(pos_es).intersection(set(neg_es))
#print(both_ess)
print("essential in both conditions: ", len(both_ess))
#what genes are essential in pos but not neg?
es_only_pos = list(set(pos_es)-set(neg_es))
#print(es_only_pos)
print("essential in positive only: ", len(es_only_pos))
es_only_neg = list(set(neg_es)-set(pos_es))
#print(es_only_neg)
print("essential in negative only: ", len(es_only_neg))

#ga and es together
pos_es = pos_genes[0] + pos_genes[1]
neg_es = neg_genes[0] + neg_genes[1]
both_ess = set(pos_es).intersection(set(neg_es))
#print(both_ess)
print("number of gd and essential genes in both conditions: ", len(both_ess))
#what genes are essential in pos but not neg?
es_only_pos = list(set(pos_es)-set(neg_es))
#print(es_only_pos)
print("gd or essential in positive only: ", len(es_only_pos))
es_only_neg = list(set(neg_es)-set(pos_es))
#print(es_only_neg)
print("gd or essential in negative only: ", len(es_only_neg))

essential in both conditions:  375
essential in positive only:  108
essential in negative only:  80
number of gd and essential genes in both conditions:  513
gd or essential in positive only:  137
gd or essential in negative only:  83


There are a different set of essential genes in each condition with positive having 28 more essential genes total. Adding in GD genes doesn't particularly improve (positive has slightly more GD genes than negative)

Will this be more coherent with different normalisation? or with only one experiment? 

### transit on nonorm data with all replicates together

In [11]:
# run transit on nonorm data (allow transit to pool from wigs as replicates)
!transit hmm output/A1_insertions.wig,output/A2_insertions.wig,output/C1_insertions.wig,output/C2_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table transit_hmm_pos_ttr.txt
!transit hmm output/B1_insertions.wig,output/B2_insertions.wig,output/D1_insertions.wig,output/D2_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table transit_hmm_neg_ttr.txt

#must give entire filename with extension or file will not be found, also this does not work with combined wigs--only takes first column

[hmm] Starting HMM Method
[hmm] Getting Data
[hmm] Normalizing using: TTR
[hmm] Combining Replicates as 'Mean'
[hmm] Running HMM Method... 99.9%    
[hmm] Finished HMM - Sites Method
[hmm] Adding File: transit_hmm_pos_ttr
[hmm] Creating HMM Genes Level Output
[hmm] Adding File: _genes.transit_hmm_pos_ttr
[hmm] Finished HMM Method
[hmm] Starting HMM Method
[hmm] Getting Data
[hmm] Normalizing using: TTR
[hmm] Combining Replicates as 'Mean'
[hmm] Running HMM Method... 99.9%    
[hmm] Finished HMM - Sites Method
[hmm] Adding File: transit_hmm_neg_ttr
[hmm] Creating HMM Genes Level Output
[hmm] Adding File: _genes.transit_hmm_neg_ttr
[hmm] Finished HMM Method


In [16]:
#how do essential genes compare (naive comparison) with all reps together with ttr norm in transit only
import pandas as pd
def read_transit(transit_genes_file):   
    with open(transit_genes_file) as f:
        data = pd.read_csv(f, sep='\t', header=None, comment='#')
        data.columns = ['ORF', 'gene', 'annot', 'TAs', 'ES_sites', 'GD_sites', 'NE_sites', 'GA_sites', 'saturation', 'mean', 'call']
        es_genes = data[data['call'] == 'ES']
        es_orfs  = es_genes['ORF'].tolist()
        gd_genes = data[data['call'] == 'GD']
        gd_orfs  = gd_genes['ORF'].tolist()
        ne_genes = data[data['call'] == 'NE']
        ne_orfs  = ne_genes['ORF'].tolist()
        ga_genes = data[data['call'] == 'GA']
        ga_orfs  = ga_genes['ORF'].tolist()
    return es_orfs, gd_orfs, ne_orfs, ga_orfs

pos_genes = read_transit('output/transit/transit_hmm_pos_ttr_genes.txt')
neg_genes = read_transit('output/transit/transit_hmm_neg_ttr_genes.txt')

pos_es = pos_genes[0] 
neg_es = neg_genes[0]

#how many genes are essential in both?
both_ess = set(pos_es).intersection(set(neg_es))
#print(both_ess)
print("Number of genes essential in both conditions: ", len(both_ess))
#what genes are essential in pos but not neg?
es_only_pos = list(set(pos_es)-set(neg_es))
#print(es_only_pos)
print("Essential in positive only: ", len(es_only_pos))
es_only_neg = list(set(neg_es)-set(pos_es))
#print(es_only_neg)
print("essential in negative only: ", len(es_only_neg))

Number of genes essential in both conditions:  375
Essential in positive only:  108
essential in negative only:  80


In [14]:
#run with sum instead of mean since some reps are sparse
# run transit on nonorm data (allow transit to pool from wigs as replicates)
!transit hmm output/A1_insertions.wig,output/A2_insertions.wig,output/C1_insertions.wig,output/C2_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table output/transit_hmm_pos_sum.txt -r Sum
!transit hmm output/B1_insertions.wig,output/B2_insertions.wig,output/D1_insertions.wig,output/D2_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table output/transit_hmm_neg_sum.txt -r Sum


[hmm] Starting HMM Method
[hmm] Getting Data
[hmm] Normalizing using: TTR
[hmm] Combining Replicates as 'Sum'
[hmm] Running HMM Method... 99.9%    
[hmm] Finished HMM - Sites Method
[hmm] Adding File: output/transit_hmm_pos_sum.txt
[hmm] Creating HMM Genes Level Output
[hmm] Adding File: output/transit_hmm_pos_sum_genes.txt
[hmm] Finished HMM Method
[hmm] Starting HMM Method
[hmm] Getting Data
[hmm] Normalizing using: TTR
[hmm] Combining Replicates as 'Sum'
[hmm] Running HMM Method... 99.9%    
[hmm] Finished HMM - Sites Method
[hmm] Adding File: output/transit_hmm_neg_sum.txt
[hmm] Creating HMM Genes Level Output
[hmm] Adding File: output/transit_hmm_neg_sum_genes.txt
[hmm] Finished HMM Method


In [17]:

import pandas as pd
def read_transit(transit_genes_file):   
    with open(transit_genes_file) as f:
        data = pd.read_csv(f, sep='\t', header=None, comment='#')
        data.columns = ['ORF', 'gene', 'annot', 'TAs', 'ES_sites', 'GD_sites', 'NE_sites', 'GA_sites', 'saturation', 'mean', 'call']
        es_genes = data[data['call'] == 'ES']
        es_orfs  = es_genes['ORF'].tolist()
        gd_genes = data[data['call'] == 'GD']
        gd_orfs  = gd_genes['ORF'].tolist()
        ne_genes = data[data['call'] == 'NE']
        ne_orfs  = ne_genes['ORF'].tolist()
        ga_genes = data[data['call'] == 'GA']
        ga_orfs  = ga_genes['ORF'].tolist()
    return es_orfs, gd_orfs, ne_orfs, ga_orfs

pos_genes = read_transit('output/transit/transit_hmm_pos_sum_genes.txt')
neg_genes = read_transit('output/transit/transit_hmm_neg_sum_genes.txt')

pos_es = pos_genes[0] 
neg_es = neg_genes[0]

#how many genes are essential in both?
both_ess = set(pos_es).intersection(set(neg_es))
#print(both_ess)
print("Number of genes essential in both conditions: ", len(both_ess))
#what genes are essential in pos but not neg?
es_only_pos = list(set(pos_es)-set(neg_es))
#print(es_only_pos)
print("Essential in positive only: ", len(es_only_pos))
es_only_neg = list(set(neg_es)-set(pos_es))
#print(es_only_neg)
print("essential in negative only: ", len(es_only_neg))

Number of genes essential in both conditions:  404
Essential in positive only:  67
essential in negative only:  106


Really get a lot fewer essential genes in positive condition using sum instead of mean, and more in negative (less sparse?)

### TRANSIT for each experiment separately


In [95]:
!transit hmm output/A1_insertions.wig,output/C1_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/transit/hmm_pos_exp1.txt
!transit hmm output/A2_insertions.wig,output/C2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/transit/hmm_pos_exp2.txt
!transit hmm output/B1_insertions.wig,output/D1_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/transit/hmm_neg_exp1.txt
!transit hmm output/B2_insertions.wig,output/D2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table output/transit/hmm_neg_exp2.txt

[hmm] Starting HMM Method
[hmm] Getting Data
[hmm] Normalizing using: TTR
[hmm] Combining Replicates as 'Mean'
[hmm] Running HMM Method... 99.9%    
[hmm] Finished HMM - Sites Method
[hmm] Adding File: output/transit/hmm_pos_exp2.txt
[hmm] Creating HMM Genes Level Output
[hmm] Adding File: output/transit/hmm_pos_exp2_genes.txt
[hmm] Finished HMM Method
[hmm] Starting HMM Method
[hmm] Getting Data
[hmm] Normalizing using: TTR
[hmm] Combining Replicates as 'Mean'
[hmm] Running HMM Method... 99.9%    
[hmm] Finished HMM - Sites Method
[hmm] Adding File: output/transit/hmm_neg_exp1.txt
[hmm] Creating HMM Genes Level Output
[hmm] Adding File: output/transit/hmm_neg_exp1_genes.txt
[hmm] Finished HMM Method
[hmm] Starting HMM Method
[hmm] Getting Data
[hmm] Normalizing using: TTR
[hmm] Combining Replicates as 'Mean'
[hmm] Running HMM Method... 99.9%    
[hmm] Finished HMM - Sites Method
[hmm] Adding File: output/transit/hmm_neg_exp2.txt
[hmm] Creating HMM Genes Level Output
[hmm] Adding File:

In [102]:
#compare experiment 1
pos_genes = read_transit('output/transit/hmm_pos_exp1_genes.txt')
neg_genes = read_transit('output/transit/hmm_neg_exp1_genes.txt')

pos_es = pos_genes[0] 
neg_es = neg_genes[0]

#how many genes are essential in both?
both_ess = set(pos_es).intersection(set(neg_es))
#print(both_ess)
print("essential in both conditions exp1: ", len(both_ess))
#what genes are essential in pos but not neg?
es_only_pos = list(set(pos_es)-set(neg_es))
#print("essential in only positive: ", es_only_pos)
print("number of essential genes in only positive: ", len(es_only_pos))
es_only_neg = list(set(neg_es)-set(pos_es))
#print(es_only_neg)
print("number of essential genes only in men neg: ", len(es_only_neg))

#compare experiment 2
pos_genes = read_transit('output/transit/hmm_pos_exp2_genes.txt')
neg_genes = read_transit('output/transit/hmm_neg_exp2_genes.txt')

pos_es = pos_genes[0] 
neg_es = neg_genes[0]

#how many genes are essential in both?
both_ess = set(pos_es).intersection(set(neg_es))
#print(both_ess)
print("essential in both conditions exp2: ", len(both_ess))
#what genes are essential in pos but not neg?
es_only_pos = list(set(pos_es)-set(neg_es))
#print("essential in only positive: ", es_only_pos)
print("number of essential genes in only positive: ", len(es_only_pos))
es_only_neg = list(set(neg_es)-set(pos_es))
#print(es_only_neg)
print("number of essential genes only in men neg: ", len(es_only_neg))


essential in both conditions exp1:  391
number of essential genes in only positive:  106
number of essential genes only in men neg:  117
essential in both conditions exp2:  344
number of essential genes in only positive:  225
number of essential genes only in men neg:  174


Experiment 2 finds a lot more essential genes in the positive condition--has much lower saturation in positive so these are presumably false positives. Also quite a few more in the men negative condition than experiment 1 where men-neg is more sparse. Using all reps seems to improve situation (==more TA sites)

How many genes really will 'become' essential in menadione? 

## Try new package 'TTN-Fitness'

Takes into account sequence context to reduce effect of non-permissive sites on GA/GD calls. This will have less effect in sub-saturated libraries anyway.

In [ ]:
#run gumbel to get ES calls
!transit gumbel output/pooled_neg.wig ref_seqs/Mbovis_LT708304.prot_table transit_gumbel_neg
!transit gumbel output/pooled_pos.wig ref_seqs/Mbovis_LT708304.prot_table transit_gumbel_pos

#run ttn-fitness to get GA/GD calls taking in to account sequence context
#!transit ttnfitness output/pooled_neg.wig ref_seqs/Mbovis_LT708304.prot_table ref_seqs/Mbovis_AF2122-97.fasta transit_gumbel_neg ttnfitness_neg_gene ttnfitness_neg_site
#!transit ttnfitness output/pooled_pos.wig ref_seqs/Mbovis_LT708304.prot_table ref_seqs/Mbovis_AF2122-97.fasta transit_gumbel_pos ttnfitness_pos_gene ttnfitness_pos_site

Gumbel method doesn't like distribution of the pooled datasets (these have been normalised with TTR before being pooled). Try with comma-separated list of wig files

In [67]:
#run with individual wig files, all together (not pooled but libraries as reps and non-normalised, will be automatically merged)
#!transit gumbel output/A1_insertions.wig,output/A2_insertions.wig,output/C1_insertions.wig,output/C2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table transit_gumbel_pos
#!transit gumbel output/B1_insertions.wig,output/B2_insertions.wig,output/D1_insertions.wig,output/D2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table transit_gumbel_neg

#run ttn-fitness to get GA/GD calls taking in to account sequence context
!transit ttnfitness output/A1_insertions.wig,output/A2_insertions.wig,output/C1_insertions.wig,output/C2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table ref_seqs/Mbovis_AF2122_97.fasta transit_gumbel_pos ttnfitness_pos_gene ttnfitness_pos_site
!transit ttnfitness output/B1_insertions.wig,output/B2_insertions.wig,output/D1_insertions.wig,output/D2_insertions.wig ref_seqs/Mbovis_LT708304.prot_table ref_seqs/Mbovis_AF2122_97.fasta transit_gumbel_neg ttnfitness_neg_gene ttnfitness_neg_site

[ttnfitness] Starting TTNFitness Method
[ttnfitness] Getting Data
[ttnfitness] Getting Genome
[ttnfitness] Processing wig files
[ttnfitness] Making Fitness Estimations
[ttnfitness] 	 + Filtering ES/ESB Genes
[ttnfitness] 	 + Filtering Short Genes. Labeling as Uncertain
[ttnfitness] 	 + Fitting M1
Traceback (most recent call last):
  File "/Users/jenniferstiens/anaconda3/envs/python38/bin/transit", line 8, in <module>
    sys.exit(run_main())
  File "/Users/jenniferstiens/anaconda3/envs/python38/lib/python3.8/site-packages/pytransit/__main__.py", line 43, in run_main
    main(*args, **kwargs)
  File "/Users/jenniferstiens/anaconda3/envs/python38/lib/python3.8/site-packages/pytransit/__main__.py", line 142, in main
    methodobj.Run()
  File "/Users/jenniferstiens/anaconda3/envs/python38/lib/python3.8/site-packages/pytransit/analysis/ttnfitness.py", line 384, in Run
    results1 = sm.OLS(Y,X1).fit()
  File "/Users/jenniferstiens/anaconda3/envs/python38/lib/python3.8/site-packages/statsmo

First time run, getting warnings that some of the TA sites not matching the coordinates of the genome file? Check this with list of coordinates and fasta.

In [65]:
#do wig coordinates match ta coordinates predicted by function 'find_insertion_sites'?
import scripts.tnseq_pro as tn
import pandas as pd
bovis_fasta  = tn.open_fasta('ref_seqs/Mbovis_AF2122-97.fasta')
ta_list = tn.find_insertion_sites(bovis_fasta)
print(ta_list[1:10])
print(len(ta_list))
wig_file = pd.read_csv('output/D1_insertions.wig', sep=' ', header=0, comment='#')
wig_ta_list = wig_file['variableStep'].tolist()
print(wig_ta_list[1:10])
print(len(wig_ta_list))
mis_ta = list(set(wig_ta_list) - set(ta_list))
print(len(mis_ta))
print(mis_ta[0:25])

[72, 102, 188, 246, 333, 360, 426, 448, 471]
73465
[72, 102, 188, 246, 333, 360, 426, 448, 471]
73536
54427
[2097152, 1441793, 2359298, 3014659, 3670021, 2359304, 4063245, 2228238, 2752533, 4063254, 2883608, 2621465, 3407898, 4194333, 2359326, 1835039, 1179679, 3932193, 2621474, 2359332, 1179685, 2883623, 4063280, 1179698, 3801140]


There are different coordinates in the TA column of the finished wig files than in the TA site list. But program to make wig file uses same function to find original TA coordinates. This is consistent for all the wig files. More than 54,000 differences. checked several on artemis and do not map to TA coordinates. 

Process for making wig files:

1) make ta dictionary from genome sequence (same as above function)
2) if the start site of read is in the ta dict, add a read to value of ta dict

`if site in ta_site_dict:
    ta_site_dict[site] += 1`

this doesn't change KEY of ta_site_dict, just adds to value

3) wig file made from this dictionary

Possible reasons:
Different genome file on thoth (used H37Rv possibly?)

- same file based on notebook (could this be wrong one?)
- is it the same genome file as used for mapping?

Answer:

The genome file used on thoth is different from one used on laptop.

    on thoth:
    >LT708304.1 Mycobacterium bovis AF2122/97 genome assembly, chromosome: Mycobacterium_bovis_AF2122/97
    62144 lines

    on laptop:
    >NC_002945.3 Mycobacterium bovis AF2122/97 chromosome, complete genome
    62081 lines

    on git (from last tnseq project):
    >LT708304.1 Mycobacterium bovis AF2122/97 genome assembly, chromosome: Mycobacterium_bovis_AF2122/97
    62144 lines

The larger file is the one used for the prot tables (Mbovis_LT708304.prot_table)

As all wig files made on thoth with correct genome, just change genome file for ttnfitness command to correct one (discarded other fasta file to avoid future confusion). Resampling doesn't look at genome file, so no need to re-run analysis for that

Ran again, but seems there is a bug in the program.

File "/Users/jenniferstiens/anaconda3/envs/python38/lib/python3.8/site-packages/statsmodels/base/data.py", line 509, in _convert_endog_exog

    raise ValueError("Pandas data cast to numpy dtype of object. "
ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

Not sure it is worth the time to figure out the bug.

In [70]:
#try with experiment 1 only as better coverage
#!transit gumbel output/A1_insertions.wig,output/C1_insertions.wig ref_seqs/Mbovis_LT708304.prot_table transit_gumbel_exp1_pos
#!transit gumbel output/B1_insertions.wig,output/D1_insertions.wig ref_seqs/Mbovis_LT708304.prot_table transit_gumbel_exp1_neg
!transit ttnfitness output/A1_insertions.wig,output/C1_insertions.wig ref_seqs/Mbovis_LT708304.prot_table ref_seqs/Mbovis_AF2122_97.fasta output/gumbel_ttnfitness/transit_gumbel_exp1_pos ttnfitness_pos1_gene ttnfitness_pos1_site


[ttnfitness] Starting TTNFitness Method
[ttnfitness] Getting Data
[ttnfitness] Getting Genome
[ttnfitness] Processing wig files
[ttnfitness] Making Fitness Estimations
[ttnfitness] 	 + Filtering ES/ESB Genes
[ttnfitness] 	 + Filtering Short Genes. Labeling as Uncertain
[ttnfitness] 	 + Fitting M1
Traceback (most recent call last):
  File "/Users/jenniferstiens/anaconda3/envs/python38/bin/transit", line 8, in <module>
    sys.exit(run_main())
  File "/Users/jenniferstiens/anaconda3/envs/python38/lib/python3.8/site-packages/pytransit/__main__.py", line 43, in run_main
    main(*args, **kwargs)
  File "/Users/jenniferstiens/anaconda3/envs/python38/lib/python3.8/site-packages/pytransit/__main__.py", line 142, in main
    methodobj.Run()
  File "/Users/jenniferstiens/anaconda3/envs/python38/lib/python3.8/site-packages/pytransit/analysis/ttnfitness.py", line 384, in Run
    results1 = sm.OLS(Y,X1).fit()
  File "/Users/jenniferstiens/anaconda3/envs/python38/lib/python3.8/site-packages/statsmo

Same bug as before.

# Resampling 

can be simple like it was before, combining combined wig from each condition, or can use newer method that does permutations per TA site among the reps of a particular library and requires a string to indicate which library the reps come from in a single combined wig file.

>the permutations will be restricted to permuting counts only among samples within each library. Statistical significance is still determined from all the data in the end (by comparing the obversed difference of means between the two conditions to a null distribution). (from docs)

### Pooling experiments and/or replicates

Drawbacks: 
- sparse datasets may skew distribution as some sites will be present in all datasets and have more reads assigned to them whereas others maybe only hit in one dataset.

Advantages: 
- increased diversity of insertion sites (marginal)


Sum other replicates (as in Griffin et al, 2011). Evaluate libraries separately or use mean/sum


In [ ]:
!transit resampling -c combined_samples_TTR.wig data/metadata.txt men_neg men_pos ref_seqs/Mbovis_LT708304.prot_table resamp_all_sr.tsv -sr --ctrl_lib ABAB --exp_lib ABAB
#raise Exception("Cannot do site_restricted resampling with library strings at same time")
#Exception: Cannot do site_restricted resampling with library strings at same time


Need to do the sr resampling with each library independently. Pooling the normalised counts of the libraries not effective in this case, because then there will be no replicates.

The resampling will sum the reps or take mean, whichever is more appropriate. Evidently summing better with sparse datasets.

Do normal (gene-based) resampling with reps and different libraries. files in combined wig and metadata need to be in same order to correspond to the string. This is less sensitive than sr method.

Compare results for independent libraries with sr, and with combined libraries with standard resampling

In [ ]:
#standard resampling with combined libraries 
!transit resampling -c output/combined_samples_TTR.wig data/metadata.txt negative positive ref_seqs/Mbovis_LT708304.prot_table resamp_all.tsv --ctrl_lib AABB --exp_lib AABB

# site-restricted resampling on each library separately
!transit resampling -c output/combined_exp1_TTR.wig data/metadata_exp1.txt negative positive ref_seqs/Mbovis_LT708304.prot_table resamp_exp1.tsv -sr
!transit resampling -c output/combined_exp2_TTR.wig data/metadata_exp2.txt negative positive ref_seqs/Mbovis_LT708304.prot_table resamp_exp2.tsv -sr


### initial resampling results

Using standard method:
With all together: 5 conditionally essential genes (attenuated)

Using -sr method (supposedly more sensitive)
Exp 1 only: 11 cond ess genes
Exp 2 has 382 cond ess genes


Remove non-permissive sites before running resampling with old method may be necessary because permutations across all ta sites in gene and some are non-permissive for insertions so will introduce variation in the null distributions. It was done with in vivo analysis but isn't necessary for -sr resampling.

from dejesus, 2017
>Because nearly 10% of all TA sites in the M. tuberculosis genome match the identified nonpermissive sequence pattern, indiscriminate inclusion of these nonper- missive sites in TnSeq analyses could artificially inflate the number of predicted essential regions determined by the use of a statistical framework that assumes random integration. Indeed, this insertional bias of Himar1 may have contributed to the previous misclassification of genes in certain families, such as the MmpL and PE_PGRS genes. However, there is general agreement between our analysis of a saturated library and previous studies using subsaturated libraries (analyzed with statistical methods that assume unbiased random insertion). Thus, in subsaturated libraries, where non- permissive sites represent a smaller subset of all sites lacking insertions, the assumption of random integration appears to have a less-significant impact on essentiality analyses. *It is primarily in the context of nearly complete saturation that the insertional prefer- ences of Himar1 need to be taken into account*

>The statistical analysis that we employed in this study was a hidden Markov model
that takes advantage of both the identified sequence preference for insertion and the high level of saturation. Specifically, the HMM uses geometric distributions as likelihood functions to evaluate the read counts observed at TA sites in the entire genome, conditioning parameters based on whether the sites match the motif identified in this study.

Though HMM model which also considers TA sites individually, the HMM available doesn't have the liklihood functions for motif included.

The resampling normalises the samples--but I have already normalised in the export function to make the combined dataset. 

Try resampling with non-normalised combined file since resampling performs TTR normalisation and there would be trimming of extremes twice?

### resampling without previously normalising

In [6]:
#for all together (also above in transit section)
!transit export combined_wig output/A1_insertions.wig,output/C1_insertions.wig,output/A2_insertions.wig,output/C2_insertions.wig,output/B1_insertions.wig,output/D1_insertions.wig,output/B2_insertions.wig,output/D2_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table output/combined_samples_nonorm.wig -n nonorm

#for each experiment
!transit export combined_wig output/A1_insertions.wig,output/C1_insertions.wig,output/B1_insertions.wig,output/D1_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table output/combined_exp1_nonorm.wig -n nonorm
!transit export combined_wig output/A2_insertions.wig,output/C2_insertions.wig,output/B2_insertions.wig,output/D2_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table output/combined_exp2_nonorm.wig -n nonorm


[combined_wig] Starting Combined Wig Export
[combined_wig] Getting Data
[combined_wig] Normalizing
[combined_wig] Running Export Method...  99.3%   
[combined_wig] Finished Export
[combined_wig] Starting Combined Wig Export
[combined_wig] Getting Data
[combined_wig] Normalizing
[combined_wig] Running Export Method...  99.3%   
[combined_wig] Finished Export


In [4]:
#resampling using nonorm combined file, std resampling, all datasets
!transit resampling -c output/combined_samples_nonorm.wig data/metadata.txt negative positive\
 ref_seqs/Mbovis_LT708304.prot_table resamp_all_ttr.tsv --ctrl_lib AABB --exp_lib AABB -n TTR

# may want to change pseudocounts to 5 (-PC 5) to limit the large lfc values resulting from some datasets with low insertion counts
#but this does not change number of significant genes (and not sure if sparse datsets have low insertion counts? they have lower mean perhaps)

[resampling] site_restricted=False
[resampling] Starting resampling Method
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 4
[resampling] Time: 1439.06s
[resampling] Finished resampling Method


This seems to change each time you run it! First time, got 11 conditionally essential genes, this time only 4.

In [7]:
# site-restricted resampling on each library separately, nonorm combined files
!transit resampling -c output/combined_exp1_nonorm.wig data/metadata_exp1.txt negative positive\
 ref_seqs/Mbovis_LT708304.prot_table resamp_exp1_ttr.tsv -sr
!transit resampling -c output/combined_exp2_nonorm.wig data/metadata_exp2.txt negative positive\
 ref_seqs/Mbovis_LT708304.prot_table resamp_exp2_ttr.tsv -sr


[resampling] site_restricted=True
[resampling] Starting resampling Method
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 3
[resampling] Time: 1177.64s
[resampling] Finished resampling Method
[resampling] site_restricted=True
[resampling] Starting resampling Method
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 383
[resampling] Time: 1151.58s
[resampling] Finished resamp

With feeding in nonnormalised all datasets to resampling (which then normalises with TTR), went from 5 to 11 conditionally expressed genes. This seems arbitrary as changes when you run multiple times (final result 4).

Exp 1 much like results of all together (3) and exp 2 (383 vs 382 originally). 

This seems like experiment 2 alone is giving a lot of false positives. See insertion_analysis.Rmd for comparison of insertion densities in the libraries. Experiment 2 has > 10X greater loss in diversity between men - and men + conditions than does experiment 1.


With Experiment 1 alone (using -sr) get the same number of conditionally essential genes as with libraries together.


In [10]:
#resampling using nonorm combined file, std resampling, all datasets, loess method (-l)
!transit resampling -c output/combined_samples_nonorm.wig data/metadata.txt negative positive\
 ref_seqs/Mbovis_LT708304.prot_table resamp_all_ttr_loess.tsv --ctrl_lib AABB --exp_lib AABB -n TTR -l


[resampling] site_restricted=False
[resampling] Starting resampling Method
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Performing LOESS Correction
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Performing LOESS Correction
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 4
[resampling] Time: 1460.46s
[resampling] Finished resampling Method


## Remove non-permissive sites from analysis and run resampling

In [1]:
#iterate removing non-permissive sites

def iterate_remove_np_sites(wig_dir, outdir, fasta, np_motif='SGNTANCS'):
    
    import scripts.non_permissive as non
    import os
    import glob
    import re
    #find all np sites in fastq
    no_sites = non.find_np_sites(np_motif, 3, fasta)
    wigfiles = glob.glob(wig_dir + "/*_insertions.wig")
    for wig in wigfiles:
        new_file = non.remove_np_sites(wig, no_sites, outdir)
        print(new_file)


if __name__ == "__main__":  
    np_motif_med = 'SGNTANCS'
    bovis_fasta = "ref_seqs/Mbovis_AF2122_97.fasta"
    output_dir = "output/permissive_wigs"
    iterate_remove_np_sites("output", output_dir, bovis_fasta, np_motif_med)
    

<_io.TextIOWrapper name='output/permissive_wigs/perm_A1_insertions.wig' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='output/permissive_wigs/perm_Mb24_insertions.wig' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='output/permissive_wigs/perm_A2_insertions.wig' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='output/permissive_wigs/perm_D2_insertions.wig' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='output/permissive_wigs/perm_B1_insertions.wig' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='output/permissive_wigs/perm_C2_insertions.wig' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='output/permissive_wigs/perm_D1_insertions.wig' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='output/permissive_wigs/perm_B2_insertions.wig' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='output/permissive_wigs/perm_Mb09_insertions.wig' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='output/permissive_wigs/perm_C1_insertions.wig' mode='w' encoding='UTF-8'>


In [3]:
# make list of np sites
def find_np_sites_list(motif, offset, fasta, output_file):
    """ Use motif finder to find all non-permissive TA sites in specified genome
    Input       motif               motif for non-permissive sites
                offset              number of nt before 'TA' in motif 
                fasta               fasta file for specified genome
    Output      np_sites            list of non-permissive positions

    """
    import os
    from Bio import SeqIO
    from Bio import SeqUtils

    # parse fastq file to extract sequence and convert to string
    fasta_file = os.path.expanduser(fasta)
    for record in SeqIO.parse(fasta_file, "fasta"):
        search_seq = str(record.seq)
    #search_seq = fasta
    # find matches for non-permissive motif
    matches = SeqUtils.nt_search(search_seq.upper(), motif)
    # add one to each position to make index=1 to match positions in wig file
    indexed_positions = []
    for i in range(1, len(matches)):
        new_position = matches[i] + 1 + offset
        indexed_positions.append(new_position)
    print(len(indexed_positions))
    # create name of new file
    np_file = output_file
    # open outfile and put in comment
    outfile = open(os.path.expanduser(np_file), 'w')
    outfile.write('\n'.join(map(str, indexed_positions)))
    outfile.close()
    return indexed_positions

if __name__ == '__main__':
    np_motif_med = 'SGNTANCS'
    bovis_fasta = "ref_seqs/Mbovis_AF2122_97.fasta"
    np_sites_file = "data/bovis_np_sites.txt"
    find_np_sites_list(np_motif_med, 3, bovis_fasta, np_sites_file)

6605


In [3]:
#create combined wig for the permissive wigs (no initial normalisation)
!transit export combined_wig output/permissive_wigs/perm_A1_insertions.wig,output/permissive_wigs/perm_C1_insertions.wig,output/permissive_wigs/perm_A2_insertions.wig,output/permissive_wigs/perm_C2_insertions.wig,output/permissive_wigs/perm_B1_insertions.wig,output/permissive_wigs/perm_D1_insertions.wig,output/permissive_wigs/perm_B2_insertions.wig,output/permissive_wigs/perm_D2_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table output/permissive_wigs/combined_samples_perm.wig -n nonorm


#resample with non-permissive sites removed (not using loess correction)
!transit resampling -c output/permissive_wigs/combined_samples_perm.wig data/metadata_perm.txt\
 negative positive ref_seqs/Mbovis_LT708304.prot_table resamp_all_perm_ttr.tsv --ctrl_lib AABB --exp_lib AABB -n TTR


[combined_wig] Starting Combined Wig Export
[combined_wig] Getting Data
[combined_wig] Normalizing
[combined_wig] Running Export Method...  98.6%   
[combined_wig] Finished Export
[resampling] site_restricted=False
[resampling] Starting resampling Method
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 3
[resampling] Time: 1412.00s
[resampling] Finished resampling Method


In [1]:
#resample with non-permissive sites removed (not using loess correction)
#run second time

!transit resampling -c output/permissive_wigs/combined_samples_perm.wig data/metadata_perm.txt\
 negative positive ref_seqs/Mbovis_LT708304.prot_table resamp_all_perm_ttr2.tsv --ctrl_lib AABB --exp_lib AABB -n TTR

[resampling] site_restricted=False
[resampling] Starting resampling Method
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 5
[resampling] Time: 1419.21s
[resampling] Finished resampling Method


Resample each experiment separately with non-permissive sites removed. This is not useful except as comparison as the -sr method will eliminate need to do this when using single library.

Removing non-permissive sites in site-restricted resampling isn't necessary as permutations only occur between sites so correcting for non-permissive insertion bias but does removing the sites change the results? 

In [8]:

#create combined wig for the permissive wigs (no initial normalisation)
!transit export combined_wig output/permissive_wigs/perm_A1_insertions.wig,output/permissive_wigs/perm_C1_insertions.wig,output/permissive_wigs/perm_B1_insertions.wig,output/permissive_wigs/perm_D1_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table output/permissive_wigs/combined_exp1_perm.wig -n nonorm
!transit export combined_wig output/permissive_wigs/perm_A2_insertions.wig,output/permissive_wigs/perm_C2_insertions.wig,output/permissive_wigs/perm_B2_insertions.wig,output/permissive_wigs/perm_D2_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table output/permissive_wigs/combined_exp2_perm.wig -n nonorm

[combined_wig] Starting Combined Wig Export
[combined_wig] Getting Data
[combined_wig] Normalizing
[combined_wig] Running Export Method...  98.6%   
[combined_wig] Finished Export
[combined_wig] Starting Combined Wig Export
[combined_wig] Getting Data
[combined_wig] Normalizing
[combined_wig] Running Export Method...  98.6%   
[combined_wig] Finished Export


In [1]:
#resample with non-permissive sites removed (using site-restricted method) 
!transit resampling -c output/permissive_wigs/combined_exp1_perm.wig data/metadata_perm_exp1.txt\
 negative positive ref_seqs/Mbovis_LT708304.prot_table resamp_exp1_perm_ttr.tsv -n TTR -sr
!transit resampling -c output/permissive_wigs/combined_exp2_perm.wig data/metadata_perm_exp2.txt\
 negative positive ref_seqs/Mbovis_LT708304.prot_table resamp_exp2_perm_ttr.tsv -n TTR -sr

[resampling] site_restricted=True
[resampling] Starting resampling Method
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 11
[resampling] Time: 1144.06s
[resampling] Finished resampling Method
[resampling] site_restricted=True
[resampling] Starting resampling Method
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 383
[resampling] Time: 1354.97s
[resampling] Finished resam

Eliminating non-permissive sites has minor effect on the number of conditionally essential ('attenuated') genes. It looks like there are only a few real differences between the two conditions.

### running site-restricted on libraries together as replicates

I'm not sure this method has the best rationale, as the libraries have such different saturation densities.

Don't indicate library string but just run site restricted on all samples as replicates of each condition. This will use site-restricted, so don't need to remove the non-permissive sites.

In [9]:
#site-restricted resampling with no library strings (all as reps) and non-permissive sites removed
!transit resampling -c output/combined_samples_nonorm.wig data/metadata.txt negative positive ref_seqs/Mbovis_LT708304.prot_table resamp_pooled_sr.tsv -n TTR -sr

[resampling] site_restricted=True
[resampling] Starting resampling Method
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 19
[resampling] Time: 1263.66s
[resampling] Finished resampling Method


## transit has corplot function that uses R

Requires a matrix of mean counts per gene--generated by export mean_counts command. Do this to double-check format to use in R corrplot package. Generate spreadsheet for convenience and make plot in R

In [29]:
!transit export mean_counts output/combined_samples_nonorm.wig ref_seqs/Mbovis_LT708304.prot_table output/combined_gene_means.txt -c

ARGS=['output/combined_samples_nonorm.wig', 'ref_seqs/Mbovis_LT708304.prot_table', 'output/combined_gene_means.txt']
KWARGS={'c': True}
[mean_counts] Starting Gene Mean Counts Export
[mean_counts] Getting Data
[mean_counts] Normalizing
[mean_counts] Running Export Method... 100.0%   79.8%   
[mean_counts] Finished Export


## use mann-whitney U-test

This is less sensitive to outliers.

>This is a rank-based test on whether the level of insertions in a gene or chromosomal region are significantly higher or lower in one condition than the other. Effectively, the insertion counts at the TA sites in the region are pooled and sorted. Then the combined ranks of the counts in region A are compared to those in region B, and p-value is calculated that reflects whether there is a significant difference in the ranks. The advantage of this method is that it is less sensitive to outliers (a unusually high insertion count at just a single TA site).

Perhaps this would have been more useful for the in-vivo dataset as we had a lot of PCR jackpots.

Chao et al suggests the method is more resistant to noise (outliers) but may not be as robust (sensitive?) to genes with low insertion counts (smaller changes?)

this method is very fast

In [33]:
#transit utest <comma-separated .wig control files> <comma-separated .wig experimental files> <annotation .prot_table or GFF3> <output file> [Optional Arguments]

!transit utest output/permissive_wigs/perm_B1_insertions.wig,output/permissive_wigs/perm_D1_insertions.wig,output/permissive_wigs/perm_B2_insertions.wig,output/permissive_wigs/perm_D2_insertions.wig\
 output/permissive_wigs/perm_A1_insertions.wig,output/permissive_wigs/perm_C1_insertions.wig,output/permissive_wigs/perm_A2_insertions.wig,output/permissive_wigs/perm_C2_insertions.wig\
 ref_seqs/Mbovis_LT708304.prot_table output/utest_all_perm_ttr.tsv -n TTR

[utest] Starting Mann-Whitney U-test Method
[utest] Getting Data
[utest] Normalizing using: TTR
[utest] Running Mann-Whitney U-test Method... 100.0%   
[utest] Performing Benjamini-Hochberg Correction
[utest] Adding File: output/utest_all_perm_ttr.tsv
[utest] Finished Mann-Whitney U-test Method


## change other parameters for resampling

-winz is supposed to reduce effect of outliers 
-PC 5 will raise pseudocount to 5 (from 1 default) and reduce high log fold changes

winsorizing in this case is just replacing highest count in each gene by second highest--another thing that would have been good to try with the in-vivo data. Not sure it's as important here.

In [34]:
!transit resampling -c output/permissive_wigs/combined_samples_perm.wig data/metadata_perm.txt\
 negative positive ref_seqs/Mbovis_LT708304.prot_table output/resamp_all_perm_winz_pc5.tsv --ctrl_lib AABB --exp_lib AABB -n TTR -winz -PC 5

[resampling] site_restricted=False
[resampling] Starting resampling Method
[resampling] Winsorizing insertion counts
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 11
[resampling] Time: 1488.30s
[resampling] Finished resampling Method


## Gene set enrichment using TRANSIT

Transit has built in functions for gene set enrichment. Annotation files will be for Mtb vs Mbovis but may be able to use to make new ones. Can choose method to rank the genes:

SLPV is signed-log-p-value (default); LFC is log2-fold-change from resampling

If we use LFC only, may need to remove genes with few sites because have v high LFC. May want to focus on new set with adjusted PC parameter which resists high LFC

Might be easier to do in R since can more easily see what's going on and don't have to worry about wrapper functions? 
Not straightforward how to use resampling results.

Can use existing pathway files as these just relate code to pathway

For Sanger: H37Rv_sanger_roles.dat (associations)               sanger_roles.dat (pathways)
For COG:    H37Rv_COG_roles.dat(associations)                   COG_roles.dat (pathways)


In [44]:
#using log fold change instead of log of pvalue (-ranking LFC) using default for exponent to use in calculating enrichment score (0)

!transit pathway_enrichment output/resamp_all_perm_winz_pc5.tsv GSEA_files/Mbovis_sanger_roles.dat GSEA_files/sanger_roles.dat output/gsea_sanger.txt -M GSEA -ranking LFC -p 1 -LFC_col 6 -Pval_col 10 -Qval_col 11
 

[pathway_enrichment] Starting Pathway Enrichment Method


No significant enrichment when using 'SLPV' (signed log of p-value) for ranking. 3 pathways enriched when using LFC

```
# significant pathways enriched for conditionally ESSENTIAL genes: 3 (qval<0.05, mean_rank<2022) (includes genes that are MORE required in condition B than A)
#   IV.K Chelatases (mean_rank=30.0)
#   I.F.2 Pyrimidine ribonucleotide biosynthesis (mean_rank=1666.8)
#   I.B.8 ATP-proton motive force (mean_rank=1747.4)
```

IV.K	Chelatases	2	30.0	0.991838	0.001225	0.048606	MB2875c/NA (26) MB0983/NA (34)

Ranks of other pathways so close to mean, not sure they are relevant

I.F.2	Pyrimidine ribonucleotide biosynthesis	9	1666.8	0.859638	0.000500	0.029753	MB1420/pyrF (62) MB1725/pyrG (1177) MB1418/carA (1793) MB1419/carB (1794) MB1415/pyrB (1993) MB2163/pyrD (2013) MB1416/pyrC (2020) MB0389c/pyrE (2040) MB2907c/pyrH (2109)

I.B.8	ATP-proton motive force	8	1747.4	0.950825	0.000000	0.000000	MB1340/atpA (180) MB1336/atpB (1799) MB1338/atpF (1817) MB1342/atpD (1904) MB1337/atpE (1926) MB1343/atpC (2039) MB1341/atpG (2102) MB1339/atpH (2212)



In [50]:
#using COG

!transit pathway_enrichment output/resamp_all_perm_winz_pc5.tsv GSEA_files/Mbovis_cog_roles.dat GSEA_files/COG_roles.dat output/gsea_cog.txt -M GSEA -ranking LFC -p 1 -LFC_col 6 -Pval_col 10 -Qval_col 11 -p 1


[pathway_enrichment] Starting Pathway Enrichment Method


NO enrichment for COG terms using either LFC or SLPV with either 0 or 1 as exponent factor

In [51]:
#with MFET method (fisher's exact test)
#not likely to work as very few genes meet pvalue threshold 
!transit pathway_enrichment output/resamp_all_perm_winz_pc5.tsv GSEA_files/Mbovis_GO_terms.dat GSEA_files/GO_term_names.dat output/gsea_fet.txt -M FET -Pval_col 10 -Qval_col 11


[pathway_enrichment] Starting Pathway Enrichment Method
[pathway_enrichment] Adding File: output/gsea_fet.txt
[pathway_enrichment] Finished Pathway Enrichment Method


As expected, no significant enrichment within the 11 statistically significant genes

Attempt to dowload KEGG gene associations. 

https://www.genome.jp/kegg/rest/keggapi.html

In [62]:
#!curl -i https://rest.kegg.jp/list/organism
#this lists mbovis "mbo", T00132

#!curl -i https://rest.kegg.jp/list/pathway/T00132 > "data/kegg_pathways_mbovis.txt" # downloads list of pathways/terms for mbovis

#!curl -i https://rest.kegg.jp/list/T00132 #lists genome (gene names etc)

!curl -i https://rest.kegg.jp/link/pathway/T00132 > "data/kegg_mappings_mbovis.txt" # downloads list of pathways/genes for mbovis

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  122k    0  122k    0     0  54854      0 --:--:--  0:00:02 --:--:-- 54854


In [4]:
#use gsea with transit with kegg mappings

!transit pathway_enrichment output/resamp_all_perm_winz_pc5.tsv GSEA_files/kegg_gene_terms.dat GSEA_files/kegg_term_names.dat output/gsea_kegg_slpv.txt -M GSEA -ranking SLPV -p 1 -LFC_col 6 -Pval_col 10 -Qval_col 11 -p 1


[pathway_enrichment] Starting Pathway Enrichment Method


Nothing really enriched with LFC except for tRNA genes and pretty low ranked. Nothing enriched using SLPV.

for LFC:

significant pathways enriched for conditionally NON-ESSENTIAL genes: 1 (qval<0.05, mean_rank>2022) (includes genes that are LESS required in condition B than A)

mbo00970 Aminoacyl-tRNA biosynthesis (mean_rank=2104.5)


mbo00970	Aminoacyl-tRNA biosynthesis	52	2104.5	0.619602	0.000000	0.000000	GLYV/NA (1642) PHEU/NA (1782) ASPT/NA (1784) GLNU/NA (1803) SERV/NA (1807) LEUV/NA (1808) SERX/NA (1811) SERT/NA (1831) SERU/NA (1841) GLUT/NA (1843) LEUT/NA (1845) PROT/NA (1847) ARGW/NA (1851) ILET/NA (1856) ASNT/NA (1857) ALAU/NA (1876) METU/NA (1877) LYST/NA (1894) TYRT/NA (1906) GLUU/NA (1914) THRT/NA (1915) TRPT/NA (1941) ARGT/NA (1957) LYSU/NA (1958) PROY/NA (1962) MB1324/argS (1964) ARGV/NA (1971) VALU/NA (1994) ALAT/NA (1996) THRV/NA (2005) GLNT/NA (2011) MB1563/ileS (2059) METT/NA (2065) MB1715/tyrS (2070) MB1676/pheS (2077) METV/NA (2092) MB1677/pheT (2093) VALT/NA (2115) LEUW/NA (2118) THRU/NA (2135) PROU/NA (2137) HIST/NA (2143) LEUU/NA (2164) CYSU/NA (2176) ARGU/NA (2194) GLYT/NA (2196) ALAV/NA (2305) MB1441/fmt (2813) MB0042/leuS (2868) VALV/NA (3473) LEUX/NA (3852) GLYU/NA (3941)

INcidentally, Got similar results when ran using GSEA on R with separate positive lfc values alone


Want to look at whether there is a chance of getting hits in longer genes (with more TA sites) if we eliminate insertions in N/C terminal parts of gene.

Use 5% cutoff

In [1]:
!transit resampling -c output/permissive_wigs/combined_samples_perm.wig data/metadata_perm.txt\
 negative positive ref_seqs/Mbovis_LT708304.prot_table output/resampling/resamp_all_perm_winz_pc5_i5.tsv --ctrl_lib AABB --exp_lib AABB -n TTR -winz -PC 5 -iN 5 -iC 5

[resampling] site_restricted=False
[resampling] Starting resampling Method
[resampling] Winsorizing insertion counts
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 18
[resampling] Time: 1320.58s
[resampling] Finished resampling Method


In [2]:
# with 5% only at C-term
!transit resampling -c output/permissive_wigs/combined_samples_perm.wig data/metadata_perm.txt\
 negative positive ref_seqs/Mbovis_LT708304.prot_table output/resampling/resamp_all_perm_winz_pc5_iC5.tsv --ctrl_lib AABB --exp_lib AABB -n TTR -winz -PC 5 -iC 5

[resampling] site_restricted=False
[resampling] Starting resampling Method
[resampling] Winsorizing insertion counts
[resampling] Getting Data
[resampling] Preprocessing Ctrl data...
[resampling] Normalizing using: TTR
[resampling] Preprocessing Exp data...
[resampling] Normalizing using: TTR
[resampling] Running Resampling Method... 100.0%   
[resampling] Performing Benjamini-Hochberg Correction
[resampling] Number of significant conditionally essential genes (Padj<0.05): 10
[resampling] Time: 1315.98s
[resampling] Finished resampling Method
